In [ ]:
import torchdata.datapipes as dp
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import numpy as np
import ot
import matplotlib.pyplot as plt
import timeit
import copy
from math import sqrt

from torch.autograd import grad as torch_grad

In [ ]:
device = torch.device('cpu')

noise_size = 62

# Number of training epochs using classical training
num_epochs = 20

# Number of iterations of Chen training
num_Chen_iters = 5000

# 'Adam' of 'RMSProp'
which_optimizer = 'Adam'

# Learning rate for optimizers
lrG = 0.000005
lrD = 0.00005

# Beta1 hyperparam for Adam optimizers
beta1 = 0

beta2 = 0.99

ngpu = 0

# To keep the criterion Lipschitz
weight_cliping_limit = 0.01

# for gradient penalty
gp_weight = 10.0

batch_size = 1024

test_batch_size = 16384

w_dim = 4

a_dim = int(w_dim*(w_dim - 1)//2)

# if 1 use GAN1, if 2 use GAN2, etc.
which_model = 2

# slope for LeakyReLU
leakyReLU_slope = 0.2

# this gives the option to rum the training process multiple times with differently initialised GANs
#num_trials = 1

num_tests_for2d = 8

In [ ]:
# calculate T and M

def generate_signs(n: int):
    lst = []
    for i in range(2**n):
        binary_exp = list(bin(i)[2:])
        lst.append((n-len(binary_exp))*[0]+binary_exp)

    res = 2*np.array(lst, dtype=float) - np.ones((2**n,n), dtype=float)
    return res

signs = generate_signs(w_dim)

first_dim = []
second_dim = []
third_dim = []
fourth_dim = []
values = []
M_list = []

for s in range(len(signs)):
    idx = 0
    M_row = []
    for i in range(w_dim):
        for j in range(i+1,w_dim):
            first_dim.append(s)
            second_dim.append(idx)
            third_dim.append(i)
            fourth_dim.append(j)
            values.append(-1*signs[s,j].item())
            first_dim.append(s)
            second_dim.append(idx)
            third_dim.append(j)
            fourth_dim.append(i)
            values.append(signs[s,j].item())
            idx+=1
            M_row.append(signs[s,j].item() * signs[s,i].item())
    M_list.append(M_row)

indices = [first_dim,second_dim,third_dim,fourth_dim]
T = torch.sparse_coo_tensor(indices=indices,values=values, size = (len(signs),a_dim,w_dim,w_dim)).to_dense()

M = torch.tensor(M_list).unsqueeze(1)


# A function that takes W, H and B (B is the Levy Area of the Brownian Bridge) and computes A = WTH+MB
# where the hell is maribor anyway?
def wthmb(w_in: torch.Tensor, h_in: torch.Tensor, b_in: torch.Tensor):
    assert w_in.shape == (batch_size,a_dim) and h_in.shape == (batch_size, w_dim) and b_in.shape == (batch_size, a_dim)
    _W = w_in.view(1,1,batch_size,w_dim)
    _H = h_in.view(batch_size,w_dim,1)
    _B = b_in.view(1,batch_size,a_dim)
    WT = torch.matmul(_W, T).permute(0,2,1,3)
    WTH = torch.matmul(WT, _H).squeeze()
    MB = torch.mul(M,_B)
    return torch.flatten(WTH + MB, start_dim=0,end_dim=1)

